<a href="https://colab.research.google.com/github/greyhound101/IEEE-CIS-Fraud/blob/master/autoencoder_scaled.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install tensorflow==2.1.0-rc0

     |████████████████████████████████| 402.3MB 20kB/s 
     |████████████████████████████████| 450kB 45.5MB/s 
     |████████████████████████████████| 3.8MB 44.1MB/s 
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
pip install keras==2.3.1

     |████████████████████████████████| 378kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5


In [0]:
!kaggle competitions download -c ieee-fraud-detection

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [0]:
import os
os.environ['KAGGLE_USERNAME'] = "tapaskd123" # username from the json file
os.environ['KAGGLE_KEY'] = "c147d874a8706a6a8d429eb3d7ea1682" # key from the json file
!kaggle competitions download -c ieee-fraud-detection

  0% 0.00/3.21M [00:00<?, ?B/s]
100% 3.21M/3.21M [00:00<00:00, 108MB/s]
 79% 41.0M/52.2M [00:00<00:00, 26.2MB/s]
100% 52.2M/52.2M [00:01<00:00, 51.3MB/s]
 70% 41.0M/58.3M [00:01<00:01, 14.8MB/s]
100% 58.3M/58.3M [00:01<00:00, 31.1MB/s]
  0% 0.00/3.26M [00:00<?, ?B/s]
100% 3.26M/3.26M [00:00<00:00, 201MB/s]
  0% 0.00/1.14M [00:00<?, ?B/s]
100% 1.14M/1.14M [00:00<00:00, 159MB/s]


In [0]:
import pandas as pd
import numpy as np
trn_trans=pd.read_csv('train_identity.csv.zip')
trn=pd.read_csv('train_transaction.csv.zip')
tst_trans=pd.read_csv('test_identity.csv.zip')
tst=pd.read_csv('test_transaction.csv.zip')
fnl_trn=trn.merge(trn_trans,on='TransactionID',how='left')
fnl_tst=tst.merge(tst_trans,on='TransactionID',how='left')
fnl_trn=fnl_trn.drop(list(fnl_trn.filter(regex='V')),1)
fnl_tst=fnl_tst.drop(list(fnl_tst.filter(regex='V')),1)
import gc
dk={}
for i in range(1,39):
    if i<10:
        dk['id-0'+str(i)]='id_0'+str(i)
    else:
        dk['id-'+str(i)]='id_'+str(i)
fnl_tst=fnl_tst.rename(columns=dk)
del([trn,tst,trn_trans,tst_trans])
gc.collect()

FileNotFoundError: ignored

In [0]:
fnl_trn['is_train']=1
fnl_tst['is_train']=0
fnl=pd.concat([fnl_trn,fnl_tst],0)
fnl=fnl.reset_index(drop=True)
gc.collect()
numerical = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
categorical = ["ProductCD", "card1", "card2", "card3", "card4", "card5", "card6", "addr1", "addr2",
               "P_emaildomain", "R_emaildomain",
              "DeviceInfo", "DeviceType"] + ["id_0" + str(i) for i in range(1, 10)] +\
                ["id_" + str(i) for i in range(10, 39)] + \
                 ["M" + str(i) for i in range(1, 10)]
for col in numerical:
    fnl[col]=fnl[col].fillna(fnl[col].mean())
for col in categorical:
    fnl[col]=fnl[col].fillna('nan')

fnl['days']=fnl['TransactionDT']//86400
fnl['id']=fnl['days']-fnl['D1']
fnl['id']=fnl['id'].astype(str)+fnl['id'].astype(str)+fnl['card1'].astype(str)+fnl['addr1'].astype(str)+fnl['ProductCD'].astype(str)
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
fnl['id']=le.fit_transform(fnl['id'])
from tqdm import tqdm
for col in tqdm(numerical):
    try:
        fnl[col+'_mean']=fnl.groupby(['id'])[col].transform('mean')
        fnl[col+'_std']=fnl.groupby(['id'])[col].transform('std')
    except:
        continue
fnl=fnl.drop(['id'],1)
from tqdm import tqdm
for col in tqdm(categorical):
        print('before '+str(fnl[col].nunique()))
        fnl.loc[~fnl[col].isin(list(fnl[col].value_counts().index[:5])),col]='other'
        print('before '+str(fnl[col].nunique()))
        if col!='card1' or col !='id_02':
            fnl_dum=pd.get_dummies(fnl[col]).astype('uint8')
            fnl_dum.columns=list(np.char.add(col,np.asarray(list(fnl_dum)).astype(str)))
            fnl_dum.columns=fnl_dum.columns+'_dum'
            
            #fnl=fnl.drop([col],1)
            fnl=pd.concat([fnl,fnl_dum],1)
fnl['day']=fnl['TransactionDT']//86400
fnl['month']=fnl['day']//30
fnl['week']=fnl['day']//7

NameError: ignored

In [0]:
import lightgbm as lgb
import gc
fnl.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in fnl.columns]
#fnl=fnl.drop(['TransactionID','TransactionDT'],1)
fnl=fnl.reset_index(drop=True)
gc.collect()
fnl=fnl.fillna(-1)
ls=set(list(fnl))
# fnl=fnl[kps]


In [0]:
tot=['day','month','week','is_train','isFraud']
ls=list(fnl.filter(regex='dum'))
tot+=ls
ls=list(fnl.select_dtypes(include=object))
tot+=ls

tot=fnl[tot]
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
ls=list(fnl.drop(tot,1))
fnl=pd.DataFrame(ss.fit_transform(fnl.drop(tot,1)))
fnl.columns=ls
fnl=pd.concat([fnl,tot],1)

In [0]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        df[col]=df[col].fillna(-1)
        df[col]=df[col].replace([np.inf,-np.inf],-1)
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df
#fnl.columns=ls
#fnl=reduce_mem_usage(fnl)
# tst=pd.read_csv('test_transaction.csv.zip',usecols=['TransactionDT'])
# trn=pd.read_csv('train_transaction.csv.zip',usecols=['TransactionDT'])
# sdo1=pd.concat([trn,tst],0)
# sdo1['day']=sdo1['TransactionDT']//86400
# sdo1['week']=sdo1['day']//7
# sdo1['month']=sdo1['day']//30
# sdo1=sdo1.reset_index(drop=True)
# fnl=pd.concat([fnl,sdo1.drop(['TransactionDT'],1),tot],1)
# del([sdo1,trn,tst,tot])
# gc.collect()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
drive.mount("/content/gdrive", force_remount=True)
import pandas as pd


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Mounted at /content/gdrive


In [0]:
#fnl=fnl.drop(['isFraud'],1)
trn=pd.read_csv('train_transaction.csv.zip',usecols=['isFraud'])
fnl['isFraud']=trn['isFraud']
#fnl=pd.concat([fnl,tot],1)
fnl=fnl.loc[:,~(fnl.columns.duplicated())]
fnl.to_csv(path)

KeyboardInterrupt: ignored

In [0]:
fnl=fnl.loc[:,~(fnl.columns.duplicated())]

In [0]:
from tqdm import tqdm
import gc
pds=['day','month','week']
#cols=list(fnl)

cols = ["TransactionAmt", "dist1", "dist2"] + ["C" + str(i) for i in range(1, 15)] + \
            ["D" + str(i) for i in range(1, 16)] 
for pd in pds:
  for col in tqdm(cols):
    col=str(col)
    use_col=str(col)+'_'+pd+'_ss'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('max')
    fnl['b']=fnl.groupby([pd])[col].transform('min')
    fnl[use_col]=fnl[col]-fnl['b']/(fnl['a']-fnl['b'])
    use_col=str(col)+'_'+pd+'_norm'
#    if use_col in cols:
    fnl['a']=fnl.groupby([pd])[col].transform('std')
    fnl['b']=fnl.groupby([pd])[col].transform('mean')
    fnl[use_col]=(fnl[col]-fnl['a'])/fnl['b']
  
  try:
      fnl=reduce_mem_usage(fnl)
  except:
    continue
del([fnl['a'],fnl['b']])
gc.collect()

100%|██████████| 32/32 [00:51<00:00,  2.76s/it]


Memory usage of dataframe is 2235.11 MB


  0%|          | 0/32 [00:00<?, ?it/s]

Memory usage after optimization is: 754.47 MB
Decreased by 66.2%


100%|██████████| 32/32 [00:26<00:00,  1.12it/s]


Memory usage of dataframe is 888.40 MB


100%|██████████| 32/32 [00:30<00:00,  1.01it/s]


Memory usage of dataframe is 1022.34 MB


568

In [0]:
ls=[]
for col in cols:
    ls+=list(fnl.filter(regex=col))
len(ls)
fnl=fnl.replace([np.inf,-np.inf],0)

In [0]:

import pandas as pd
from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
trn=pd.DataFrame(ss.fit_transform(fnl[ls]))
trn.columns=ls
fnl=fnl.drop(ls,1)
fnl=pd.concat([fnl,trn],1)

In [0]:
trn_name='nn_preped.pkl'
path = F"/content/gdrive/My Drive/{trn_name}" 
fnl.to_pickle(path)

In [0]:
fnl.shape

(1097231, 751)

In [0]:
import gc
import os
gc.collect()
import numpy as np
import keras 
from math import ceil
from keras.models import Model
import matplotlib.pyplot as plt
from keras.utils import Sequence
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Concatenate, Dropout,BatchNormalization
from keras.layers import Add
import keras.backend as K
class DAESequence(Sequence):
    def __init__(self,df,batch_size):
        self.batch_size=batch_size
        self.len_data=df.shape[0]
        self.columns=df.shape[1]
        self.data=df.values
        self.idx=[]
    def __len__(self):
        return int(ceil(self.len_data/self.batch_size))
    def __getitem__(self,idx):
        self.idx.append(idx)
        last=min(idx+self.batch_size,self.len_data)
        size=last-idx
        output_x=self.data[idx:last]
        length=last-idx
        rnd=np.random.randint(0,self.len_data-size,size)
        rnd[rnd>idx]+=size
        cols=np.random.randint(0,self.columns,int(self.columns*0.1))
        noise_x=output_x.copy()
        noise_x[:,cols]=self.data[rnd[:,None],cols]
        return noise_x,output_x
from keras.models import Sequential
from keras.callbacks.callbacks import ReduceLROnPlateau,ModelCheckpoint
from keras.layers import BatchNormalization
#fnl=pd.concat([fnl_trn,fnl_tst],0).select_dtypes(exclude='uint8')
batch_size=1024
checkpoint_path = "training_1.hdf5" 
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback =ModelCheckpoint(checkpoint_path, 
monitor='loss',save_best_only=True,save_weights_only=True,verbose=1)


Using TensorFlow backend.


In [0]:
len_input_columns=fnl.shape[1]
kernel_initializer_1=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_0=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.04499212706658476, seed=None)
kernel_initializer_2=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_3=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_4=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_5=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_6=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_7=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
kernel_initializer_8=keras.initializers.RandomNormal(mean=-4.74564e-04, stddev=0.031814238148788886, seed=None)
model_dae = Sequential()
model_dae.add(Dense(units=len_input_columns,input_shape=(len_input_columns,), activation='relu', dtype='float32', name='Input'))
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_1', kernel_initializer=kernel_initializer_0))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_2', kernel_initializer=kernel_initializer_1))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_3', kernel_initializer=kernel_initializer_2))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_4', kernel_initializer=kernel_initializer_3))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_5', kernel_initializer=kernel_initializer_4))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_6', kernel_initializer=kernel_initializer_5))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_7', kernel_initializer=kernel_initializer_6))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_8', kernel_initializer=kernel_initializer_7))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=int(100), activation='relu', dtype='float32',name='Hidden_9', kernel_initializer=kernel_initializer_8))
model_dae.add(BatchNormalization())
model_dae.add(Dense(units=len_input_columns, activation='linear', dtype='float32', name='Output'))
model_opt = keras.optimizers.SGD(lr=0.001, decay=1-0.9, momentum=0.7, nesterov=False)
model_dae.compile(loss='mean_squared_error',optimizer=model_opt)
model_dae.load_weights('/content/gdrive/My Drive/autoenc.hdf5')
# es = callbacks.EarlyStopping(monitor='val_loss', min_delta=0.00000000000000000001, patience=30,verbose=1, mode='auto', restore_best_weights=True)
#     # Callback for Reducing the Learning Rate... when the monitor levels out for 'patience' epochs, then the LR is reduced
# rlr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,patience=2, min_lr=1e-10, mode='auto')

ValueError: ignored